# 1. PCA for Return Allocation

## 1.0. Importing Libraries

In [59]:
"""START OF PCA LIBRARIES"""
# Load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import read_csv, set_option
from pandas.plotting import scatter_matrix
import seaborn as sns
from sklearn.preprocessing import StandardScaler

#Import Model Packages
from numpy.linalg import inv, eig, svd
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.decomposition import KernelPCA
"""END OF PCA LIBRARIES"""

'END OF PCA LIBRARIES'

## 1.1. Importing and Preparing the Data

In [60]:
# Import packages and read the data
import pandas as pd

nyse_df = pd.read_csv("NYSE_prices_split_adjusted.csv")
nyse_df.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [61]:
# Remove unnecessary columns
labels = ["open", "low", "high", "volume"]
nyse_df.drop(labels=labels, inplace=True, axis=1)
nyse_df.head()

,date,symbol,close
0,2016-01-05,WLTW,125.839996
1,2016-01-06,WLTW,119.980003
2,2016-01-07,WLTW,114.949997
3,2016-01-08,WLTW,116.620003
4,2016-01-11,WLTW,114.970001


In [62]:
# Pivot the dataframe
nyse_df = nyse_df.pivot(index="date", columns="symbol", values="close")
nyse_df.head()

symbol,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XLNX,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,22.389128,4.77,40.380001,30.572857,NaN,26.629999,26.129884,42.070000,37.090000,31.670000,...,25.379999,69.150002,35.330002,8.63,NaN,17.100000,25.226457,60.020000,13.33,NaN
2010-01-05,22.145923,5.31,40.139999,30.625713,NaN,26.440001,25.918773,42.330002,37.700001,31.620001,...,25.059999,69.419998,34.910000,8.64,NaN,17.230000,25.140187,61.919998,13.80,NaN
2010-01-06,22.067240,5.09,40.490002,30.138571,NaN,26.190001,26.062713,42.779999,37.619999,31.559999,...,24.889999,70.019997,35.139999,8.56,NaN,17.170000,24.960462,61.900002,15.00,NaN
2010-01-07,22.038626,5.24,40.480000,30.082857,NaN,25.770000,26.278623,42.740002,36.889999,31.309999,...,24.639999,69.800003,35.599998,8.60,NaN,16.700001,24.953272,63.320000,16.68,NaN
2010-01-08,22.031474,5.14,40.639999,30.282858,NaN,26.049999,26.412967,42.570000,36.689999,31.490000,...,25.000000,69.519997,35.599998,8.57,NaN,16.700001,24.960462,61.990002,16.41,NaN


## 1.2. Descriptive Statistics

In [63]:
nyse_df.describe()

symbol,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XLNX,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZION,ZTS
count,1762.000000,1762.000000,1762.000000,1762.000000,1008.000000,1762.000000,1762.000000,1762.000000,1762.000000,1762.000000,...,1762.000000,1762.000000,1762.000000,1762.000000,1008.000000,1762.000000,1762.000000,1762.000000,1762.000000,987.000000
mean,34.080221,23.873768,103.769597,79.402683,55.473641,60.957747,34.628491,75.211884,55.247911,46.349637,...,39.094001,84.006481,44.558087,9.992565,36.477470,27.291992,48.223388,82.873859,25.069489,40.142432
std,7.439467,15.557106,43.410717,28.330794,8.690668,25.974059,8.184385,23.328453,25.523074,11.165272,...,8.346671,10.255660,9.657220,1.823279,6.763168,11.406846,10.432182,24.229881,5.020573,7.749647
min,19.291845,4.000000,39.160000,27.435715,33.709999,25.770000,21.667749,36.439999,22.690001,26.500000,...,23.280001,56.570000,27.780001,6.230000,24.190001,11.090000,23.522646,46.970001,13.330000,28.400000
25%,27.687768,9.332500,66.465000,55.450000,49.315000,37.939999,25.973951,56.637501,32.619999,37.615001,...,32.619999,79.202497,36.572501,8.600000,33.345000,15.920000,39.234365,60.842500,21.350000,32.169998
50%,34.332382,17.225000,84.584999,78.435711,56.389999,56.060000,35.849998,75.020001,45.720000,46.274999,...,39.509998,86.005001,42.369999,9.940000,36.205000,26.750000,50.413373,79.549999,25.140000,42.369999
75%,40.559486,39.349998,148.100003,103.092503,62.309999,81.165001,41.454999,93.000000,76.289999,55.225000,...,46.097499,90.430000,51.985002,11.080000,38.179998,37.377500,54.879586,103.482500,28.879999,47.145001
max,48.439999,55.759998,200.380005,133.000000,71.230003,115.480003,51.200001,125.400002,110.809998,74.529999,...,61.240002,104.379997,65.139999,14.320000,54.750000,52.369999,68.209925,133.089996,43.930000,55.380001


In [64]:
nyse_df.shape

(1762, 501)

## 1.3. Data Cleaning

In [65]:
# Checking for any null values and removing the null values
print('Null Values =',nyse_df.isnull().values.any())

Null Values = True


In [66]:
# Removing stocks that have more than 30% NaN values
nyse_df2 = nyse_df.copy()

missing_fractions = nyse_df2.isnull().mean().sort_values(ascending=False)

drop_list = sorted(list(missing_fractions[missing_fractions > 0.3].index))

nyse_df2.drop(labels=drop_list, axis=1, inplace=True)
nyse_df2.shape

(1762, 470)

I decided to not go through with removing stocks with >30% NaN values. It removed all stocks from the df that were founded after ~2012, including (but not limited to) some big names with a potentially large impac such as: Google (Alphabet), PayPal, and Mondelez. Instead, due to the narrow time frame of this dataset, all NaN values will be back filled, so a company added in 2012 @ $35.68 will read $35.68 up until that point. It will also take care of any random NaN values that occur later in the set and not effect the data too much.

In [67]:
# Fill the missing values with the last value available in the dataset.
nyse_df = nyse_df.fillna(method="bfill")
nyse_df.head()

symbol,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XLNX,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,22.389128,4.77,40.380001,30.572857,35.119999,26.629999,26.129884,42.070000,37.090000,31.670000,...,25.379999,69.150002,35.330002,8.63,27.5,17.100000,25.226457,60.020000,13.33,31.01
2010-01-05,22.145923,5.31,40.139999,30.625713,35.119999,26.440001,25.918773,42.330002,37.700001,31.620001,...,25.059999,69.419998,34.910000,8.64,27.5,17.230000,25.140187,61.919998,13.80,31.01
2010-01-06,22.067240,5.09,40.490002,30.138571,35.119999,26.190001,26.062713,42.779999,37.619999,31.559999,...,24.889999,70.019997,35.139999,8.56,27.5,17.170000,24.960462,61.900002,15.00,31.01
2010-01-07,22.038626,5.24,40.480000,30.082857,35.119999,25.770000,26.278623,42.740002,36.889999,31.309999,...,24.639999,69.800003,35.599998,8.60,27.5,16.700001,24.953272,63.320000,16.68,31.01
2010-01-08,22.031474,5.14,40.639999,30.282858,35.119999,26.049999,26.412967,42.570000,36.689999,31.490000,...,25.000000,69.519997,35.599998,8.57,27.5,16.700001,24.960462,61.990002,16.41,31.01


In [68]:
# Daily Linear Returns (%)
nyse_dailyReturns = nyse_df.pct_change(1)
print(nyse_dailyReturns)

nyse_dailyReturns2 = nyse_dailyReturns.copy()
# Remove Outliers beyond 3 standard deviation
nyse_dailyReturns2 = nyse_dailyReturns2[nyse_dailyReturns2.apply(lambda x :(x-x.mean()).abs()<(3*x.std()) ).all(1)]
print(nyse_dailyReturns2)

symbol             A       AAL       AAP      AAPL      ABBV       ABC   
date                                                                     
2010-01-04       NaN       NaN       NaN       NaN       NaN       NaN  \
2010-01-05 -0.010863  0.113208 -0.005944  0.001729  0.000000 -0.007135   
2010-01-06 -0.003553 -0.041431  0.008720 -0.015906  0.000000 -0.009455   
2010-01-07 -0.001297  0.029470 -0.000247 -0.001849  0.000000 -0.016037   
2010-01-08 -0.000325 -0.019084  0.003953  0.006648  0.000000  0.010865   
...              ...       ...       ...       ...       ...       ...   
2016-12-23  0.005003 -0.004313  0.002934  0.001978  0.011028  0.009639   
2016-12-27  0.006710  0.002682  0.005559  0.006351  0.002567  0.003310   
2016-12-28 -0.016986 -0.019338 -0.008263 -0.004264 -0.003680 -0.005963   
2016-12-29 -0.001750 -0.008811 -0.000821 -0.000257  0.007387  0.012636   
2016-12-30 -0.001753 -0.011852 -0.006812 -0.007796 -0.001754 -0.014495   

symbol           ABT       ACN      A

I decided against removing data outside of 3 standard deviations as it resulted in a drastic loss of data (1762x501 to 217x501). These outliers could be genuine data points caused by real events (market crashes, stock splitting, etc) rather than human error. Removing these could have a drastic impact on the performance of the PCA we are conducting, as such I have decided to leave them in.

## 1.4. Data Transformation